# Introduction to `rasterio` and `atlite`

In this tutorial, we're going to cover two new Python packages:

**Rasterio: access to geospatial raster data**

> Geographic information systems use GeoTIFF and other formats to organize and store gridded raster datasets such as satellite imagery and terrain models. Rasterio reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON.

:::{note}
Documentation for rasterio is available at https://rasterio.readthedocs.io/en/latest/index.html.
:::

**Atlite: Convert weather data to energy systems data**

> Atlite is a free software, xarray-based Python library for converting weather data (such as wind speeds, solar radiation, temperature and runoff) into power systems data (such as wind power, solar power, hydro power and heating demand time series).

:::{note}
Documentation for atlite is available at https://atlite.readthedocs.io/en/latest/.

This tool is developed here at TU Berlin in the group of Digital Transformation of Energy Systems.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install atlite rasterio geopandas cartopy matplotlib numpy xarray
```
:::



## Download of Required Data

Before we can run this tutorial, we have to download a few input data files to your local filesystem. The cell below uses some Python utility packages to automate the retrieval of the required files.

In [ ]:
from urllib.request import urlretrieve
from os.path import basename

url = (
    "https://tubcloud.tu-berlin.de/s/2oogpgBfM5n4ssZ/download/PORTUGAL-2013-01-era5.nc"
)
urlretrieve(url, basename(url))

## Repetition of Vector Data

In the last tutorial, we used `geopandas` to work with **vector data** (e.g. points, lines, polygons):

Importing the package:

In [ ]:
import geopandas as gpd

Reading the file:

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/7bpHrAkjMT3ADSr/download/country_shapes.geojson"

In [ ]:
countries = gpd.read_file(url).set_index("name")

Inspecting the GeoDataFrame:

In [ ]:
countries.tail()

:::{note}
This dataset uses ISO-2 country codes as index: https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2.
:::

Plotting the GeoDataFrame:

In [ ]:
countries.plot(edgecolor="k", facecolor="lightgrey")

Reprojecting geometries in the GeoDataFrame:

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
crs = ccrs.EqualEarth()

fig = plt.figure(figsize=(10, 5))

ax = plt.axes(projection=crs)

countries.to_crs(crs.proj4_init).plot(ax=ax, edgecolor="k", facecolor="lightgrey")

## Raster Data with `rasterio`

But we haven't worked with **raster data** yet!

One useful package for working with raster data is [`rasterio`](https://rasterio.readthedocs.io/en/latest/):

In [ ]:
import rasterio

With `rasterio`, we can for instance load GeoTIFF files.

In the example below, we load the [CORINE land cover classification](https://land.copernicus.eu/pan-european/corine-land-cover) for Portugal:



In [ ]:
url = "https://tubcloud.tu-berlin.de/s/Mw5dwbwdsDY8zfH/download/U2018_CLC2018_V2020_20u1-PT.tif"
clc = rasterio.open(url)

Datasets can have one or more bands (or layers), which can be read with the `.read()` function:

In [ ]:
band = clc.read(1)

The returned object is a 2-dimensional `numpy.ndarray`. 

In [ ]:
type(band)

In [ ]:
band.shape

The band values correspond to a land cover and land use classification scheme with class values as listed [here](https://collections.sentinel-hub.com/corine-land-cover/readme.html).

This is an example of **categorical values** in raster data.

In [ ]:
import numpy as np

In [ ]:
np.unique(band)

There are two common options to plot raster data:
- with `plt.imshow()`
- with `rasterio.plot.show()`

In [ ]:
plt.imshow(band, cmap="tab20")

In [ ]:
from rasterio.plot import show

show(band, cmap="tab20")

The integrated `rasterio` plotting functionality is slightly more powerful.

For instance, it can label the axes with proper geo-referenced extents.

These are stored in the `.crs`, `.bounds` and `.transform` attributes.

In [ ]:
clc.crs

In [ ]:
clc.bounds

In [ ]:
clc.transform

We can pass the information on the projection to the `rasterio.plot.show()` function:

In [ ]:
from rasterio.plot import show

show(band, transform=clc.transform, cmap="tab20")

Let's quickly load another dataset with **continuous values**: The elevation and water depth in and around Portugal.

This dataset comes from [GEBCO Gridded Bathymetry Data](https://www.gebco.net/data_and_products/gridded_bathymetry_data/) and has another file format `.nc` (netCDF) which also stores raster data.

It would be useful to exclude very mountainous areas, or identify shallow water at sea for the development of offshore wind turbines.

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/XoDpBcweJHmYKgF/download/GEBCO_2014_2D-PT.nc"
gebco = rasterio.open(url)

In [ ]:
band = gebco.read(1)

In [ ]:
np.unique(band)

For plotting, we can also display **vector and raster data together**:

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8))

countries.loc[["PT", "ES"]].plot(ax=ax, color="none")

show(band, transform=gebco.transform, cmap="RdBu_r", ax=ax)

## Land Eligibility Analysis with `atlite`

To estimate the wind and solar generation potential of a country, land usage constraints need to be taken into account.

Data required to determine the eligible areas is stored as **raster** and **vector** data.

**Examples:**

- **Vector Data:** natural protection areas, settlements, roads, airports
- **Raster Data:** type of land, elevation, slope

The [`atlite`](https://atlite.readthedocs.io) library has some useful features to handle exclusion constraints for both datatypes.

The core object to do land elibility analysis in `atlite` is the `ExclusionContainer`, which serves as a container for all rasters and geometries we want to exclude (or include).

In [ ]:
from atlite.gis import ExclusionContainer

In many cases, rasters and geometries have different projections and resolutions. Therefore, the `ExclusionContainer` is initialized by a CRS and a resolution which suits as a basis for all added rasters and geometries. Per default the CRS is [3035](https://epsg.io/3035) and the resoultion 100, which leads set a raster of 100 meter resolution. All rasters and geometries will be converted to this CRS and resolution.

:::{warning}
The [CRS 3035](https://epsg.io/3035) is only valid in Europe.
:::

In [ ]:
excluder = ExclusionContainer(crs=3035)

Let's add our first exclusion criteria, the [Natura2000 natural protection areas](https://www.google.com/search?channel=fs&client=ubuntu&q=natura200), to the container using the `excluder.add_geometry()` function. This function is for vector data. It's sibling, `excluder.add_raster()` we will use later.

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/mxgaA7wH8NsmdDi/download/Natura2000_end2021-PT.gpkg"
excluder.add_geometry(url)

To calculate the *mask* for the exclusion container (i.e. which cells are not excluded), we can use the `shape_availability()` function, which takes as arguments:
- a **geometry** (e.g. country shape) within which to calculate available areas
- an **exclusion container** object

In [ ]:
from atlite.gis import shape_availability

Since the excluder is in `crs=3035`, we should also convert to geometry of Portucal to `excluder.crs` first.

In [ ]:
shape = countries.to_crs(excluder.crs).loc[["PT"]].geometry
shape[0]

We can then call the `shape_availability` function:

In [ ]:
band, transform = shape_availability(shape, excluder)

This function returns a 2-D `numpy.ndarray` and a transformation, similar to the previous `rasterio` examples.

Eligile raster cells have a `True` and excluded cells a `False`.

In [ ]:
band

In [ ]:
band.any()

In [ ]:
transform

We can use the `rasterio.plot.show()` function for plotting:

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8))
shape.plot(ax=ax, color="none")
show(band, transform=transform, cmap="Greens", ax=ax)

How to calculate what share of the land is eligible?
1. count number of eligible raster cells
2. determine area per raster cell
3. multiply cell area with number of cells
4. divide eligible area by total land area

In [ ]:
eligible_cells = band.sum()

In [ ]:
cell_area = excluder.res**2

In [ ]:
eligible_area = cell_area * eligible_cells

In [ ]:
country_area = shape.geometry.area[0]

In [ ]:
eligible_area / country_area * 100

In this example, approximately 20% of Portugal's land area are covered by nature reserves.

Typically, more than one dataset is needed for a proper land eligibility analysis, i.e. we need to add more datasets to our container.
So, let's expand the exclusion container ...

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/mxgaA7wH8NsmdDi/download/Natura2000_end2021-PT.gpkg"
excluder = ExclusionContainer()
excluder.add_geometry(url)

... by a selection of CORINE land cover classes, which is given as raster data:

When adding a raster to the `ExclusionContainer` you can specify which **codes** (integers) to exclude.

By setting `invert=True`, you can also restrict the inclusion to a set of codes.

Furthermore, you can add a **buffer** around codes which should also be excluded.

Here we are first going to include a set of suitable [CORINE classes](https://collections.sentinel-hub.com/corine-land-cover/readme.html):

In [ ]:
fn = "https://github.com/fneum/data-science-for-esm/raw/main/data-science-for-esm/U2018_CLC2018_V2020_20u1-PT.tif"

In [ ]:
codes = [12, 13, 18, 19, 20, 21]

In [ ]:
excluder.add_raster(fn, codes=codes, crs=3035, invert=True)

And then we are going exclude [CORINE classes](https://collections.sentinel-hub.com/corine-land-cover/readme.html) representing settled areas with an 800 meter buffer around them:

In [ ]:
codes = [1, 2, 3, 4, 5, 6]

In [ ]:
excluder.add_raster(fn, codes=codes, buffer=800, crs=3035)

Then, we can simply repeat the calculation and visualisation of available cells:

In [ ]:
band, transform = shape_availability(shape, excluder)

fig, ax = plt.subplots(figsize=(4, 8))
shape.plot(ax=ax, color="none")
show(band, transform=transform, cmap="Greens", ax=ax)

## Spatial and Temporal Availability

Besides calculating spatial potential for renewables development, [`atlite`](https://atlite.readthedocs.io) can also handle weather data time series and convert them to energy systems time series (like production time series for wind and solar).

One famous such dataset is the [ERA5 reanalysis data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) which has historical weather data for the whole world from 1959 to 2022 at hourly resolution and on a 0.25 x 0.25 degree grid (roughly 25-30 km depending on where you are).

`atlite` can be imported like this (usually without an alias):

In [ ]:
import atlite

A cutout is the basis for any of your work and calculations and contains all the weather data.

Here, we load an existing dataset which was retrieved from `ERA5`.

In [ ]:
cutout = atlite.Cutout("PORTUGAL-2013-01-era5.nc")

In [ ]:
cutout

In [ ]:
cutout.data

Let's quickly plot the spatial resolution and scope of this dataset:

In [ ]:
ax = shape.to_crs(4326).plot()
cutout.grid.plot(ax=ax, edgecolor="grey", color="None")

We can directly access, aggregate, and plot individual meteorological parameters:

In [ ]:
wnd100m = cutout.data.wnd100m.mean(dim="time")

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

wnd100m.plot(ax=ax, vmin=0, vmax=10)

shape.to_crs(4326).plot(ax=ax, edgecolor="k", color="none")

This cutout happens to also have data on the surface roughness. We can use this to scale the 100 meter wind speeds down to 10 meters using the **log law**:

$$\frac{U(z)}{U(z_r)} = \frac{\ln\left(\frac{z}{z_0}\right)}{\ln\left(\frac{z_r}{z_0}\right)}$$

In [ ]:
z_r = cutout.data.roughness
wnd100m = cutout.data.wnd100m

In [ ]:
wnd10m = wnd100m * np.log(10 / z_r) / np.log(100 / z_r)

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

wnd10m.mean(dim="time").plot(ax=ax, vmin=0, vmax=10)

shape.to_crs(4326).plot(ax=ax, edgecolor="k", color="none")

We can also build means in the spatial dimensions and investigate the behaviour of meteorological variables over time. Here, we do this for the direct solar irradiation.

In [ ]:
cutout.data.mean(dim=["x", "y"]).influx_direct.plot()

We also already saw that the weather cells are much larger than the raster cells. `atlite` automates this conversion for averaging fine-grained to coarse-grained rasters for all geometries in shapes when calling the `cutout.availabilitymatrix()` function. 

In [ ]:
A = cutout.availabilitymatrix(shape, excluder)

`A` is an array with 3 dimensions (`shape`, `x`, `y`) and very sparse data. It indicates the relative overlap of weather cell (`x`, `y`) with geometry shape while excluding the area specified by the excluder.

In [ ]:
fig, ax = plt.subplots()
A.sel(name="PT").plot(cmap="Greens")
shape.to_crs(4326).plot(ax=ax, edgecolor="k", color="none")
cutout.grid.plot(ax=ax, color="none", edgecolor="grey", ls=":")

The availibility matrix `A` can now be used as a layout matrix in the conversion functions of `atlite` from wind speeds to capacity factors.

The normal approach would be to weigh the availabilities with the area per grid cell and the capacity per area.

:::{note}
No worries, if this part of the code is a bit too complicated! We might touch on `xarray` a little later in the course.
:::

In [ ]:
import xarray as xr

cap_per_sqkm = 2
area = cutout.grid.set_index(["y", "x"]).to_crs(3035).area / 1e6
area = xr.DataArray(area, dims=("spatial"))

capacity_matrix = A.stack(spatial=["y", "x"]) * area * cap_per_sqkm

We can now calculate the capacity factor time series for Portugal.

In [ ]:
cutout.prepare()
wind = cutout.wind(matrix=capacity_matrix, turbine="Vestas_V90_3MW", index=shape.index)

Finally, let’s see how the total wind power potential per region look.

In [ ]:
wind.plot()

## Exercises

Tweak some parameters of the land exclusion calculation:

- vary the CORINE land classes allower for wind turbine development
- vary the buffer distance and observe the change in availble area
- add the GEBCO topography data to the exclusion calculator and exclude areas above 1000 m